In [11]:
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sys
import os
import logging
import random
import copy
import time
import queue
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

sys.path.append(os.path.abspath(".."))
from Modules.Helper import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotters
from Modules.Equation import Equation

In [12]:
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='../Data/GRN5_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])
t_eval = np.array(df['t'])
original = np.array(df[labels]).T

bounds = {
    'tau': (0.1, 5.0),
    'k': (0.1, 2.0),
    'n': (0, 30.0)
}

IND_SIZE = 19
GENERATION_LIMIT = 5
NO_IMPROVEMENT_LIMIT = 50
HARD_SIGMA_INCREASE_FACTOR = 10
TOLERANCE = 1E-4

In [13]:
class Individual:
    def __init__(self, method='RK45', error='ABS'):
        # Estrutura do indivíduo contendo os coeficientes para cada variável
        self.coeffs = {
            'A': {
                'E': {'n': None, 'k': None, '-': True},
                'tau': None
            },
            'B': {
                'A': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'C': {
                'B': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'E': {
                'D': {'n': None, 'k': None, '-': False},
                'B': {'n': None, 'k': None, '-': False},
                'E': {'n': None, 'k': None, '-': False},
                'tau': None,
            }
        }
        
        self.ind_size = IND_SIZE
        self.fitness = np.inf
        self.method = method
        self.error = error
        
    @staticmethod
    def get_bounds_list():
        bounds_list = []
        gene_regulators = {
            'A': ['E'],
            'B': ['A'],
            'C': ['B'],
            'D': ['C'],
            'E': ['D', 'B', 'E']
        }
        for gene in ['A', 'B', 'C', 'D', 'E']:
            bounds_list.append(bounds['tau'])
            for _ in gene_regulators[gene]:
                bounds_list.append(bounds['n'])
                bounds_list.append(bounds['k'])
        return bounds_list
     
    @staticmethod
    def list_to_ind(list_ind, method, error):
        i = 0
        ind = Individual(method=method, error=error)
        
        # Mapeamento dos reguladores para cada gene
        gene_regulators = {
            'A': ['E'],
            'B': ['A'],
            'C': ['B'],
            'D': ['C'],
            'E': ['D', 'B', 'E']  # Assumindo 3 reguladores para E
        }
        
        for gene in ['A', 'B', 'C', 'D', 'E']:
            # Atribui tau
            ind.coeffs[gene]['tau'] = list_ind[i]
            i += 1
            
            # Atribui n e k para cada regulador
            for regulator in gene_regulators[gene]:
                ind.coeffs[gene][regulator]['n'] = list_ind[i]
                ind.coeffs[gene][regulator]['k'] = list_ind[i+1]
                i += 2
                
        return ind

    def ind_to_list(self):
        ind_list = []
        for gene in ['A', 'B', 'C', 'D', 'E']:
            ind_list.append(self.coeffs[gene]['tau'])
            
            regulators = [k for k in self.coeffs[gene].keys() if k != 'tau']
            for regulator in regulators:
                ind_list.append(self.coeffs[gene][regulator]['n'])
                ind_list.append(self.coeffs[gene][regulator]['k'])
                
        return ind_list
    
    @staticmethod
    def apply_bounds(population, method, error):
        for ind in population:
            list_ind = Individual.list_to_ind(ind, method, error)
            ind[:] = Individual.ind_to_list(list_ind)
    
    @staticmethod    
    def cma_evaluate(method, error, list_ind):
        ind = Individual.list_to_ind(list_ind, method, error)
        ind.calc_fitness()
        return ind.fitness,
    
    # Calcula o fitness do indivíduo simulando o sistema de ODEs
    def calc_fitness(self):
        try:
            equation = Equation(self.numerical_coeffs, labels)
            # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
            y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
            
            if self.error == 'ABS':
                self.fitness = self.abs_error(original, y)
            elif self.error == 'MSE':
                self.fitness = self.MSE_error(original, y)
            elif self.error == 'MABS':
                self.fitness = self.mean_abs_error(original, y)
            elif self.error == 'SQUARED':
                self.fitness = self.squared_error(original, y)
            
            self.fitness = min(self.fitness, 1e6)
        except:
            # Trata exceções relacionadas ao solver
            print("Fitness Overflow")
            self.fitness = 1e6
            
    def calc_all_fitness(self):
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        
        return {
            'ABS_Fitness': self.abs_error(original, y), 
            'SQUARED_Fitness': self.squared_error(original, y),
            'MSE_Fitness': self.MSE_error(original, y), 
            'MABS_Fitness': self.mean_abs_error(original, y), 
        }
     
       
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals
        
        dA = equation.full_eq(vals, 'A', 'E')
        dB = equation.full_eq(vals, 'B', 'A')
        dC = equation.full_eq(vals, 'C', 'B')
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+B', '+D'], ['+D', '+E']])

        return [dA, dB, dC, dD, dE]
    
    @staticmethod
    def abs_error(original, pred):
        return sum(sum(abs(original-pred)))
    
    @staticmethod
    def squared_error(original, pred):
        return sum(sum( (original-pred)**2 ))**(1/2)
    
    @staticmethod
    def MSE_error(original, pred):
        return np.mean((original-pred)**2)
    
    @staticmethod
    def mean_abs_error(original, pred):
        return np.mean(abs(original-pred))
            
    @staticmethod
    def initialize_ind():
        ind = Individual()
        gene_regulators = {
            'A': ['E'],
            'B': ['A'],
            'C': ['B'],
            'D': ['C'],
            'E': ['D', 'B', 'E']
        }
        
        for gene in ['A', 'B', 'C', 'D', 'E']:
            # Gera valor para tau
            ind.coeffs[gene]['tau'] = random.uniform(*bounds['tau'])
            
            # Gera valores para n e k de cada regulador
            for regulator in gene_regulators[gene]:
                ind.coeffs[gene][regulator]['n'] = random.uniform(*bounds['n'])
                ind.coeffs[gene][regulator]['k'] = random.uniform(*bounds['k'])
        
        ind.calc_fitness()
        return ind
    
    @property
    def numerical_coeffs(self):
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for gene in numerical_coeffs:
            numerical_coeffs[gene]['tau'] = self.coeffs[gene]['tau']
            for regulator in numerical_coeffs[gene]:
                if regulator != 'tau':
                    numerical_coeffs[gene][regulator]['n'] = int(self.coeffs[gene][regulator]['n'])
                    numerical_coeffs[gene][regulator]['k'] = self.coeffs[gene][regulator]['k']
        return numerical_coeffs
    
    def plot(self, method='RK45', comparison=True):
        methods = [self.method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        Plotters.plot_methods(results=results,t=t_eval, methods=methods, labels=labels)
        if comparison:
            Plotters.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)
            
    @staticmethod        
    def initialize_average_bounds(bounds, ind_size):
        averages = [np.mean(value) for value in bounds.values()]
        array = np.resize(averages, ind_size)
        return array   
        
    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual (fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"

In [14]:
def run_evolution(seed, error, method, result_container, timeout_limit):
    logging.info(f"Starting DE execution: Solver={method}, Error={error}, Seed={seed}")
    
    bounds_list = Individual.get_bounds_list()
    np.random.seed(seed)
    
    def objective_function(params):
        ind = Individual.list_to_ind(params, method, error)
        ind.calc_fitness()
        return ind.fitness
    
    start_time = time.time()
    
    try:
        result = differential_evolution(
            objective_function,
            bounds_list,
            strategy='best1bin',
            maxiter=1000,
            popsize=15,
            mutation=0.8,
            recombination=0.9,
            seed=seed,
            polish=False,
            callback=lambda x, convergence: time.time() - start_time > timeout_limit,
            disp=True
        )
        
        best_ind = Individual.list_to_ind(result.x, method, error)
        best_ind.calc_fitness()
        
        # Debug: Mostra resultados parciais
        print(f"\nSeed {seed} ({error}) - Melhor fitness: {best_ind.fitness:.4f}")
        print("Parâmetros:", best_ind.ind_to_list())
        
    except Exception as e:
        logging.error(f"DE Error: {str(e)}")
        best_ind = None

    finally:
        execution_time = time.time() - start_time
        
        errors_result = best_ind.calc_all_fitness() if best_ind else {
            'ABS_Fitness': None, 'SQUARED_Fitness': None, 
            'MSE_Fitness': None, 'MABS_Fitness': None
        }

        result_data = {
            'best_ind': best_ind,
            'error_type': error,
            'method': method,
            'seed': seed,
            'ABS_Fitness': errors_result['ABS_Fitness'],
            'SQUARED_Fitness': errors_result['SQUARED_Fitness'],
            'MSE_Fitness': errors_result['MSE_Fitness'],
            'MABS_Fitness': errors_result['MABS_Fitness'],
            'execution_time': execution_time,
        }

        result_container.append(result_data)
        logging.info(f"Execution finished for {method}, seed {seed}, error {error} in {execution_time:.2f} seconds.")
        
CSV_FILENAME = 'results-de.csv'

if os.path.exists(CSV_FILENAME):
    results_df = pd.read_csv(CSV_FILENAME)
    existing_results = results_df[['seed', 'error_type', 'method']]
else:
    results_df = pd.DataFrame(columns=[
        'best_ind', 'error_type', 'method', 'seed', 'ABS_Fitness',
        'SQUARED_Fitness', 'MSE_Fitness', 'MABS_Fitness', 'execution_time'
    ])
    existing_results = pd.DataFrame(columns=['seed', 'error_type', 'method'])

all_methods = ['RK45']
errors = ['ABS', 'SQUARED', 'MSE', 'MABS']
seeds = 10
TIMEOUT_LIMIT = 60*60  # 1 hora

# Thread-safe queue e paralelização
result_queue = queue.Queue()
write_lock = threading.Lock()

def process_and_save_results():
    global results_df
    while True:
        result = result_queue.get()
        if result is None:
            break
            
        with write_lock:
            # Converter o indivíduo para string serializável
            if result['best_ind'] is not None:
                result['best_ind'] = str(result['best_ind'].ind_to_list())
            
            results_df = pd.concat([results_df, pd.DataFrame([result])], ignore_index=True)
            results_df.to_csv(CSV_FILENAME, index=False)

def process_seed_error_method(seed, error, method):
    if not existing_results[
        (existing_results['seed'] == seed) &
        (existing_results['error_type'] == error) &
        (existing_results['method'] == method)
    ].empty:
        return

    result_container = []
    try:
        run_evolution(seed, error, method, result_container, TIMEOUT_LIMIT)
    except Exception as e:
        logging.error(f"Critical error: {str(e)}")
    
    if result_container:
        result_queue.put(result_container[0])

# Iniciar thread de escrita
writer_thread = threading.Thread(target=process_and_save_results, daemon=True)
writer_thread.start()

# Execução paralela
with ThreadPoolExecutor(max_workers=4) as executor:  # Reduzir workers para GRN complexo
    futures = []
    for seed in range(seeds):
        for error in errors:
            for method in all_methods:
                futures.append(executor.submit(process_seed_error_method, seed, error, method))
    
    for future in as_completed(futures):
        future.result()

# Finalizar
result_queue.put(None)
writer_thread.join()

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 1: f(x)= 10.366
differential_evolution step 1: f(x)= 0.429816
differential_evolution step 1: f(x)= 114.827
differential_evolution step 2: f(x)= 8.37395
differential_evolution step 2: f(x)= 0.280492
differential_evolution step 1: f(x)= 0.459306
differential_evolution step 2: f(x)= 114.461
differential_evolution step 3: f(x)= 8.37395
differential_evolution step 3: f(x)= 0.280492
differential_evolution step 3: f(x)= 74.9034
differential_evolution step 4: f(x)= 7.73947
differential_evolution step 4: f(x)= 0.239598
differential_evolution step 2: f(x)= 0.457845
differential_evolution step 4: f(x)= 74.1731
differential_evolution step 5: f(x)= 7.73947
differential_evolution step 5: f(x)= 0.239598
differential_evolution step 3: f(x)= 0.299613
differential_evolution step 6: f(x)= 7.50162
differential_evolution step 6: f(x)= 0.225097
differential_evolution step 5: f(x)= 68.8047
differential_evolution step 7: f(x)= 7.50162
differential_evolution step 7: f(x)= 0.225097
d

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 139: f(x)= 28.4232

Seed 0 (ABS) - Melhor fitness: 28.4232
Parâmetros: [1.5855374499516564, 21.986842092590614, 0.6556626499517395, 2.256569905956903, 22.745593734472344, 0.5216558827783477, 3.3774957780305046, 8.9471855874305, 0.43705403805245835, 1.087326354997384, 4.703365826698713, 0.4719596602084978, 3.813601049473779, 13.807583253523166, 0.5276615908113941, 22.080189538314166, 0.29147533057891417, 7.536461028071758, 1.620356793060571]


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


...

differential_evolution step 147: f(x)= 2.60912

Seed 9 (SQUARED) - Melhor fitness: 2.6091
Parâmetros: [1.589732458167565, 22.760449612711493, 0.7413380242473417, 2.0712568617918654, 27.02998077823066, 0.5561597901283876, 1.2845613313295743, 16.650781806139094, 0.4619342837761342, 3.3486101195684483, 10.91394474146891, 0.2785933967506464, 3.9334537659374065, 26.02738299597837, 0.42452343111308954, 11.67788212015408, 0.25049737086568147, 11.648405158924366, 1.8592942164821984]


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 86: f(x)= 0.140155
differential_evolution step 144: f(x)= 0.0272299
differential_evolution step 87: f(x)= 0.140155
differential_evolution step 88: f(x)= 0.140155
differential_evolution step 89: f(x)= 0.140155
differential_evolution step 90: f(x)= 0.140155
differential_evolution step 91: f(x)= 0.140155
differential_evolution step 92: f(x)= 0.137496
differential_evolution step 145: f(x)= 0.0272299

Seed 9 (MSE) - Melhor fitness: 0.0272
Parâmetros: [1.589732458167565, 22.760449612711493, 0.7413380242473417, 2.0712568617918654, 27.02998077823066, 0.5561597901283876, 1.2845613313295743, 16.650781806139094, 0.4619342837761342, 3.3486101195684483, 10.91394474146891, 0.2785933967506464, 3.9334537659374065, 26.02738299597837, 0.42452343111308954, 11.67788212015408, 0.25049737086568147, 11.648405158924366, 1.8592942164821984]
differential_evolution step 93: f(x)= 0.137496


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


differential_evolution step 94: f(x)= 0.137496
differential_evolution step 95: f(x)= 0.137496
differential_evolution step 96: f(x)= 0.137496
differential_evolution step 97: f(x)= 0.137496
differential_evolution step 98: f(x)= 0.137496
differential_evolution step 99: f(x)= 0.137496
differential_evolution step 100: f(x)= 0.137496

Seed 9 (MABS) - Melhor fitness: 0.1375
Parâmetros: [1.7575004382344535, 14.01673016638598, 0.6757227537122235, 2.1224094044302246, 29.558177566992065, 0.4435944476486917, 2.0378319543552927, 14.80487603162495, 0.6081666833322606, 3.8372989484385123, 12.125470783549941, 0.5265677473618552, 4.279782844206387, 13.585670721097097, 0.15807693171092208, 27.027214692716125, 0.4416521848198991, 7.52246852910335, 1.2096120153843122]


In [15]:
logging.basicConfig(
    filename="GRN5_DE_execution.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)